In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import glacier
import glacier_v3
import cmocean
import importlib
from matplotlib import pyplot as plt, animation, rc
from IPython.display import Image
rc('animation', html='html5')
from scipy.sparse import spdiags

In [2]:
importlib.reload(glacier_v3)
ngridx=50
ngridz=20
D=200 # m
L=2e4 # m
dt= 60 
T= 1*1440*dt# 1e6 9days
alpha=0.4/86400
ML=20
zz=0 # dont use values on the second grid cell (unstable solution)
C,S,u,w=glacier_v3.glacier(ngridx,ngridz,dt,zz,T,ML,alpha,motion = True,steady=False)

In [3]:
# Plot Diffusion only:
cmap=cmocean.cm.haline
fig, axs=plt.subplots(4,1,figsize=(10,10))
z = np.linspace(0,D,ngridz+1)
x = np.linspace(0,L/1e3,ngridx+1)
X,Z=np.meshgrid(x,z)

c1=axs[0].contourf(X,-Z,C[-1,:,:].T,cmap=cmap,levels=np.arange(3,15,1))
cb1=fig.colorbar(c1,ax=axs[0],pad=0.01)
cb1.ax.get_yaxis().labelpad = 10
cb1.set_label('Methane nM', rotation=270)
c2=axs[1].contourf(X,-Z,S[-1,:,:].T,cmap=cmap,levels=np.arange(0,35,1))
cb2=fig.colorbar(c2,ax=axs[1],pad=0.01)
cb2.ax.get_yaxis().labelpad = 15
cb2.set_label('Salinity (PSU)', rotation=270)

cmapu=cmocean.cm.balance
zu = np.linspace(0,D,ngridz)
xu = np.linspace(0,L/1e3,ngridx)
Xu,Zu=np.meshgrid(xu,zu)

c3=axs[2].contourf(Xu,-Zu,u[-1,:,:].T,cmap=cmapu,levels=np.arange(-1e-1,1e-1,1e-2))
cb3=fig.colorbar(c3,ax=axs[2],pad=0.01)
cb3.ax.get_yaxis().labelpad = 15 
cb3.set_label('Velocity u (m/s)', rotation=270)

c4=axs[3].contourf(Xu,-Zu,w[-1,:,:].T,cmap=cmapu,levels=np.arange(-6e-4,6e-4,1e-5))
cb4=fig.colorbar(c4,ax=axs[3],pad=0.01)
cb4.ax.get_yaxis().labelpad = 15
cb4.set_label('Velocity w (m/s)', rotation=270)

fig.suptitle('Diffusion and Advection',y=0.95,x=0.45)
fig.supxlabel('Distance from glacier (km)',x=0.45)
fig.supylabel('Depth (m)',x=0.05)
plt.savefig('AdvDiff.pdf')

In [4]:
# Extract profiles for methane concentration with sinks, at a distance from the mouth of the glacier:
dz=D/(ngridz-1)
dx = L/(ngridx-1)
plt.figure(figsize=(5,10))
plt.plot(C[-1,7,:],-z)
#plt.axhline(y=-int(zz/dz)*dz)

In [5]:
mins,hs,ds=0,0,0
minutes,hours,days=[mins],[hs],[ds]
for t in range(int(T/60)):
    if mins < 59:
        mins+=1
        minutes.append(mins)
        hours.append(hs)
        days.append(ds)
    else:
        mins = 0
        minutes.append(mins)
        if hs< 23:
            hs+=1
            hours.append(hs)
            days.append(ds)
        else:
            hs = 0
            hours.append(hs)
            ds +=1
            days.append(ds)
time ={'m':minutes,'h':hours,'d':days}


In [6]:
fig = plt.figure(figsize=(10,10))
def animate(frames):
    if frames > T:
        factor=int(frames/T)
        frames=frames-(T*factor)
        
    dia = time['d'][int(frames)]
    hora = time['h'][int(frames)]
    minuto = time['m'][int(frames)]
    plt.clf()
    ax1 = fig.add_subplot(411)
    ax2 = fig.add_subplot(412)
    ax3 = fig.add_subplot(413)
    ax4 = fig.add_subplot(414)
    c1 = ax1.contourf(X,-Z,C[int(frames),:,:].T,cmap=cmap,levels=np.arange(3,15,1))
    cb1=fig.colorbar(c1,ax=ax1,pad=0.01)
    cb1.ax.get_yaxis().labelpad = 10
    ax1.set_title(f'{dia} days {hora}:{minuto}')
    cb1.set_label('Methane nM', rotation=270)
    c2 = ax2.contourf(X,-Z,S[int(frames),:,:].T,cmap=cmap,levels=np.arange(0,35,1))
    cb2=fig.colorbar(c2,ax=ax2,pad=0.01)
    cb2.ax.get_yaxis().labelpad = 15
    cb2.set_label('Salinity (PSU)', rotation=270)
    c3=ax3.contourf(Xu,-Zu,u[int(frames),:,:].T,cmap=cmapu,levels=np.arange(-2e-1,2e-1,1e-2))
    cb3=fig.colorbar(c3,ax=ax3,pad=0.01)
    cb3.ax.get_yaxis().labelpad = 15 
    cb3.set_label('Velocity u (m/s)', rotation=270)
    c4=ax4.contourf(Xu,-Zu,w[int(frames),:,:].T,cmap=cmapu,levels=np.arange(-1.5e-3,1.5e-3,1e-4))
    cb4=fig.colorbar(c4,ax=ax4,pad=0.01)
    cb4.ax.get_yaxis().labelpad = 15
    cb4.set_label('Velocity w (m/s)', rotation=270)
    fig.suptitle(titulo,y=0.95,x=0.45)
    fig.supxlabel('Distance from glacier (km)',x=0.45)
    fig.supylabel('Depth (m)',x=0.05)
    
    return fig
titulo=str('Glacier advection +diffusion')
anim = animation.FuncAnimation(fig, animate, frames=np.arange(0,int(T/dt)-1,120))
f = r"/Users/jvalenti/Desktop/diff.mp4" 
FFwriter = animation.FFMpegWriter()
anim.save(f, writer = FFwriter)

In [7]:
#with open(f,'rb') as anim:
#    display(Image(anim.read()))